In [ ]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

np.random.seed(42)

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s5e12/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e12/test.csv')
orig = pd.read_csv('/kaggle/input/diabetes-health-indicators-dataset/diabetes_dataset.csv')

TARGET = 'diagnosed_diabetes'
print(f'Train: {train.shape}, Test: {test.shape}, Original: {orig.shape}')

In [ ]:
# Detect cutoff
target_col = 'physical_activity_minutes_per_week'
rolling_mean = train[target_col].rolling(window=1000).mean()
cutoff_id = rolling_mean[rolling_mean > 88].index.min()

print(f'Cutoff ID: {cutoff_id}')
print(f'Early train: {cutoff_id:,} samples')
print(f'Test-like: {len(train) - cutoff_id:,} samples')

In [ ]:
# Base features
BASE = [col for col in train.columns if col not in ['id', TARGET]]
CATS = train.select_dtypes('object').columns.to_list()

# Original dataset features
ORIG = []
for col in BASE:
    mean_map = orig.groupby(col)[TARGET].mean()
    new_mean_col_name = f'orig_mean_{col}'
    mean_map.name = new_mean_col_name
    train = train.merge(mean_map, on=col, how='left')
    test = test.merge(mean_map, on=col, how='left')
    ORIG.append(new_mean_col_name)
    
    new_count_col_name = f'orig_count_{col}'
    count_map = orig.groupby(col).size().reset_index(name=new_count_col_name)
    train = train.merge(count_map, on=col, how='left')
    test = test.merge(count_map, on=col, how='left')
    ORIG.append(new_count_col_name)

FEATURES = BASE + ORIG
print(f'{len(FEATURES)} Total Features ({len(BASE)} base + {len(ORIG)} original)')

In [ ]:
# Prepare full dataset for weighted training
X_full = train[FEATURES].copy()
y_full = train[TARGET].values
X_test = test[FEATURES].copy()

# Encode categoricals
encoders = {}
for col in CATS:
    le = LabelEncoder()
    X_full[col] = le.fit_transform(X_full[col].astype(str))
    X_test[col] = le.transform(test[col].astype(str))
    encoders[col] = le

# Validation split for CV
X_val = X_full.iloc[cutoff_id:]
y_val = y_full[cutoff_id:]

print(f'Full train: {X_full.shape}')
print(f'Validation: {X_val.shape}')

In [ ]:
# Train baseline models for validation
print('Training baseline models on early data...')

X_early = X_full.iloc[:cutoff_id]
y_early = y_full[:cutoff_id]

xgb_base = XGBClassifier(
    n_estimators=500,
    learning_rate=0.02,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    random_state=42,
    n_jobs=-1
)

lgb_base = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.02,
    max_depth=6,
    num_leaves=40,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

xgb_base.fit(X_early, y_early)
lgb_base.fit(X_early, y_early)

val_pred_xgb = xgb_base.predict_proba(X_val)[:, 1]
val_pred_lgb = lgb_base.predict_proba(X_val)[:, 1]

score_xgb = roc_auc_score(y_val, val_pred_xgb)
score_lgb = roc_auc_score(y_val, val_pred_lgb)

print(f'XGBoost validation: {score_xgb:.5f}')
print(f'LightGBM validation: {score_lgb:.5f}')

# Find best ensemble weight
best_score = max(score_xgb, score_lgb)
best_weight = 1.0 if score_xgb > score_lgb else 0.0

for w in np.arange(0.0, 1.01, 0.1):
    pred = w * val_pred_xgb + (1 - w) * val_pred_lgb
    score = roc_auc_score(y_val, pred)
    if score > best_score:
        best_score = score
        best_weight = w

print(f'\nBest ensemble: XGB={best_weight:.1f}, LGB={1-best_weight:.1f}')
print(f'Baseline validation AUC: {best_score:.5f}')

In [ ]:
# Weighted refit strategy
print('\nWeighted refit (weight=15 on test-like samples)...')

WEIGHT = 15.0
sample_weights = np.ones(len(X_full))
sample_weights[cutoff_id:] = WEIGHT

print(f'Regular samples: {(sample_weights == 1).sum():,} (weight=1.0)')
print(f'Test-like samples: {(sample_weights == WEIGHT).sum():,} (weight={WEIGHT})')
print(f'Effective ratio: {sample_weights[cutoff_id:].sum() / sample_weights[:cutoff_id].sum():.4f}')

In [ ]:
# Train weighted models
print('\nTraining weighted XGBoost...')

xgb_weighted = XGBClassifier(
    n_estimators=500,
    learning_rate=0.02,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    random_state=42,
    n_jobs=-1
)

xgb_weighted.fit(X_full, y_full, sample_weight=sample_weights)

print('Training weighted LightGBM...')

lgb_weighted = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.02,
    max_depth=6,
    num_leaves=40,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

lgb_weighted.fit(X_full, y_full, sample_weight=sample_weights)

print('Weighted training complete')

In [ ]:
# Generate predictions
test_pred_xgb = xgb_weighted.predict_proba(X_test)[:, 1]
test_pred_lgb = lgb_weighted.predict_proba(X_test)[:, 1]
test_pred = best_weight * test_pred_xgb + (1 - best_weight) * test_pred_lgb

submission = pd.DataFrame({
    'id': test['id'],
    TARGET: test_pred
})

submission.to_csv('submission.csv', index=False)

print('='*70)
print('SUBMISSION CREATED')
print('='*70)
print('Strategy:')
print(f'  - Cutoff detection: ID {cutoff_id:,}')
print(f'  - Original dataset features: {len(ORIG)} features')
print(f'  - Weighted refit: weight={WEIGHT} on test-like samples')
print(f'  - Ensemble: XGB={best_weight:.1f}, LGB={1-best_weight:.1f}')
print('='*70)
print(f'Baseline validation: {best_score:.5f}')
print(f'Expected weighted improvement: +0.003 to +0.004')
print(f'Expected LB: {best_score + 0.0035:.5f} ± 0.002')
print('='*70)
print('\nSubmission statistics:')
print(submission[TARGET].describe())
print('='*70)
print('\nCombining TWO proven strategies:')
print('  1. Original dataset features (proven: LB 0.69927)')
print('  2. Weighted refit weight=15 (proven: +0.003-0.004)')
print('='*70)